CTE (Common Table Expressions)

In [ ]:
WITH vendas_recentes AS (
    SELECT cliente_id, valor
    FROM vendas
    WHERE data >= '2024-01-01'
)
SELECT cliente_id, SUM(valor) as total
FROM vendas_recentes
GROUP BY cliente_id;


NULL

In [ ]:
--- Selecting when is not null
SELECT * FROM table WHERE id ID NOT NULL

In [ ]:
--- Replacing missing values
SELECT COALESCE(price,0) AS clean_price FROM sales

In [ ]:
--- Replacing using CASE WHEN
SELECT
    CASE
        WHEN region IS NULL THEN 'Unknown'
        ELSE region
    END AS region_cleaned
FROM users

Window Function

In [ ]:
--- Row_number gives a unique row number per group
--- Use case: Get the top 1 salary per department
SELECT 
    employee_id,
    department,
    salary,
    ROW_NUMBER() OVER (PARTITION BY department ORDER BY salary DESC) AS rank_in_dept
FROM employees

In [ ]:
--- RANK / DENSE_RANK - rank with out without gaps
SELECT
    product_id,
    category,
    sales,
    RANK() OVER (PARTITION BY category ORDER BY sales DESC) AS sales_rank
FROM products

In [ ]:
--- LAG/LEAD - compare values in adjacent rows
--- Use case: Track how a customer's spending changes over time.
SELECT
    customer_id,
    purschse_date,
    amount,
    LAG(amount) OVER (PARTITION BY customer_id ORDER BY purchse_date) AS prev_purchase,
    LEAD(amount) OVER (PARTITION BY customer_id ORDER BY purchase_date) AS next_purchase
FROM purchases

In [ ]:
--- SUm/AVG/MAX 
--- Use case: Rolling totals or moving averages
SELECT 
     order_id,
     customer_id,
     order_value,
     SUM(order_value) OVER (PARTITION BY customer_id ORDER BY order_date ROWS BETWEEN 3 PRECEDING AND CURRENT ROW ) AS rolling_3_sum
FROM orders

In [ ]:
--- NTILE - break rows into buckets
--- Use case: Quartile or deciles of scores

SELECT 
    student_id,
    test_score,
    NTILE(4) OVER (ORDER BY test_score DESC) AS quartile
FROM students